# Example 2: Operation of dipole moment operator to H2O vibrational ground state

| run type | restart | wavefunction | backend | Basis  | max iteration |
| ---       | ---   | ---          | ---     | ---    | ---   |
| operation  | True (file suffix `_gs`)  | MPS-SM (restart) | Numpy   | HO-DVR | 10    |

## 1. Import modules

- Required in **any** calculations

In [1]:
from pytdscf import BasInfo, Model, Simulator

## 2. Set DVR primitive basis

**MPS-MCTDH wavefunction**
$$
|\Psi_{\rm{MPS-MCTDH}}\rangle = \sum_{\mathbf \{j\}}\sum_{\mathbf \{\tau\}}
a\substack{j_1 \\ 1\tau_1}a\substack{j_2 \\ \tau_1\tau_2} \cdots a\substack{j_f \\ \tau_{f-1}1}
|\varphi_{j_1}^{(1)}(q_1)\rangle|\varphi_{j_2}^{(2)}(q_2)\rangle
\cdots|\varphi_{j_f}^{(f)}(q_f)\rangle
$$
where SPF is 
$$
\varphi_{j_p}^{(p)}(q_p) = \sum_{i_p=1}^{n_p} c_{i_p}^{j_p}\chi_{i_p}^{(p)}(q_p) \; (j_p = 1,2,\ldots, N_p)
$$

Here, select $\{\chi_{i_p}^{(p)}(q_p)\}$ as Harmonic Oscillator eigenfunction.
See detail in [documenation](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.primints_cls.PrimBas_HO).
Here one define $n_p$ = 9, $N_p$ = 9. 

**NOTE**

- In MPS,  $n = N$ (SM) is usually better than $n < M$ (MCTDH). Only when using a laptop computer, MCTDH sometimes works better. (because the required RAM in MCTDH is smaller than SM.)

- Basis information must be the same as restart one.

In [2]:
from math import sqrt

from discvar import HarmonicOscillator

from pytdscf import units
from pytdscf.potentials.h2o_potential import k_orig
from pytdscf.dvr_operator_cls import TensorOperator
from pytdscf.hamiltonian_cls import TensorHamiltonian

import numpy as np
import sympy
from pympo import AssignManager, OpSite, SumOfProducts

backend = "numpy"

freqs = [
    sqrt(k_orig[(1, 1)]),
    sqrt(k_orig[(2, 2)]),
    sqrt(k_orig[(3, 3)]),
]  # a.u.

nprims = [9, 9, 9]  # Number of primitive basis

basis = [
    HarmonicOscillator(nprim, omega, units="a.u.")
    for nprim, omega in zip(nprims, freqs, strict=False)
]
ndim = len(basis)  # Number of degree of freedom, H2O has 3 DOFs
basinfo = BasInfo([basis])  # Set basis information object

## 3. Set Dipole Operator (Polynomial Function)

Here, one use pre-calculated Polyonimal PES and DMS.

In [3]:
from pytdscf.potentials.h2o_dipole import mu

mu  # Dipole has (x,y,z) elements

{(): [-1.69908e-15, 1.24913e-14, -1.93795],
 (3,): [-2.20831e-17, 0.00853527, -8.32759e-16],
 (2,): [1.50857e-17, 2.08217e-15, -0.00326347],
 (1,): [6.37588e-18, 8.65662e-16, 0.0142383],
 (3, 3): [3.5274e-18, -1.35302e-15, -2.31565e-05],
 (2, 3): [3.46044e-18, -0.000294259, -7.3169e-16],
 (2, 2): [-1.5306e-18, -1.42229e-15, 0.00020955],
 (1, 3): [1.45261e-17, 0.000327409, -2.99469e-17],
 (1, 2): [3.90656e-18, 1.26166e-16, -0.000112968],
 (1, 1): [-6.45481e-18, 6.79098e-16, 0.000192831],
 (3, 3, 3): [-1.34711e-21, 7.33335e-06, 9.41511e-22],
 (2, 3, 3): [2.2067e-22, -3.92968e-22, 3.0608e-06],
 (1, 3, 3): [-2.55725e-22, 4.55392e-22, -3.54702e-06],
 (2, 2, 3): [6.16547e-22, -3.35633e-06, -4.3091e-22],
 (2, 2, 2): [1.69378e-22, -3.01627e-22, 2.34936e-06],
 (1, 2, 2): [3.17065e-22, -5.64628e-22, 4.39783e-06],
 (1, 1, 3): [-1.08836e-21, 5.92476e-06, 7.60666e-22],
 (1, 1, 2): [-2.92033e-23, 5.20049e-23, -4.05062e-07],
 (1, 1, 1): [5.60185e-22, -9.97572e-22, 7.77e-06]}

### Setup one particle operator

In [4]:
q1_list = [np.array(basis_i.get_grids()) for basis_i in basis]
q2_list = [q1_int**2 for q1_int in q1_list]
q3_list = [q1_int**3 for q1_int in q1_list]
q0_list = [q1_int**0 for q1_int in q1_list]

q3_ops = [OpSite(f"q^3_{i+1}", i, value=q3_list[i]) for i in range(0, ndim)]
q2_ops = [OpSite(f"q^2_{i+1}", i, value=q2_list[i]) for i in range(0, ndim)]
q1_ops = [OpSite(f"q^1_{i+1}", i, value=q1_list[i]) for i in range(0, ndim)]
q0_ops = [OpSite(f"q^0_{i+1}", i, value=q0_list[i]) for i in range(0, ndim)]

qn_list = [q0_ops, q1_ops, q2_ops, q3_ops]

In [5]:
E = []
subs = {}

for i in range(3):
    E.append(sympy.symbols("E" + f"_{i}"))
    subs[E[i]] = 1.0

In [6]:
from collections import Counter

mu_tensor = np.zeros((4, 4, 4, 3))
key_list = list(mu.keys())

for key in key_list:
    count = Counter(key)
    for i in range(3):
        mu_tensor[count[1]][count[2]][count[3]][i] = mu[key][i]

mu_symbol = [[[[sympy.symbols("μ_{" + "1" * i + "2" * j + "3" * k + f"{l}" + "}") for l in range(3)]
               for k in range(4)]
              for j in range(4)]
             for i in range(4)]

for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(3):
                subs[mu_symbol[i][j][k][l]] = mu_tensor[i][j][k][l]

### Setup Potential Operator

In [7]:
from scipy.special import factorial

pot_sop = SumOfProducts()

for i in range(4):
    for j in range(4):
        for k in range(4):
            k_constant = 0
            k_value = 0
            for l in range(3):
                k_constant += mu_symbol[i][j][k][l] * E[l]
                k_value += mu_tensor[i][j][k][l] * subs[E[l]]
            if k_value == 0:
                continue
            pot_sop += k_constant * qn_list[i][0] * qn_list[j][1] * qn_list[k][2] / factorial(i) / factorial(j) / factorial(k)    

pot_sop = pot_sop.simplify()
pot_sop.symbol

(1.0*E_0*μ_{0} + 1.0*E_1*μ_{1} + 1.0*E_2*μ_{2})*q^0_1*q^0_2*q^0_3 + (1.0*E_0*μ_{10} + 1.0*E_1*μ_{11} + 1.0*E_2*μ_{12})*q^1_1*q^0_2*q^0_3 + (0.5*E_0*μ_{110} + 0.5*E_1*μ_{111} + 0.5*E_2*μ_{112})*q^2_1*q^0_2*q^0_3 + (0.166666666666667*E_0*μ_{1110} + 0.166666666666667*E_1*μ_{1111} + 0.166666666666667*E_2*μ_{1112})*q^3_1*q^0_2*q^0_3 + (0.5*E_0*μ_{1120} + 0.5*E_1*μ_{1121} + 0.5*E_2*μ_{1122})*q^2_1*q^1_2*q^0_3 + (0.5*E_0*μ_{1130} + 0.5*E_1*μ_{1131} + 0.5*E_2*μ_{1132})*q^2_1*q^0_2*q^1_3 + (1.0*E_0*μ_{120} + 1.0*E_1*μ_{121} + 1.0*E_2*μ_{122})*q^1_1*q^1_2*q^0_3 + (0.5*E_0*μ_{1220} + 0.5*E_1*μ_{1221} + 0.5*E_2*μ_{1222})*q^1_1*q^2_2*q^0_3 + (1.0*E_0*μ_{130} + 1.0*E_1*μ_{131} + 1.0*E_2*μ_{132})*q^1_1*q^0_2*q^1_3 + (0.5*E_0*μ_{1330} + 0.5*E_1*μ_{1331} + 0.5*E_2*μ_{1332})*q^1_1*q^0_2*q^2_3 + (1.0*E_0*μ_{20} + 1.0*E_1*μ_{21} + 1.0*E_2*μ_{22})*q^0_1*q^1_2*q^0_3 + (0.5*E_0*μ_{220} + 0.5*E_1*μ_{221} + 0.5*E_2*μ_{222})*q^0_1*q^2_2*q^0_3 + (0.166666666666667*E_0*μ_{2220} + 0.166666666666667*E_1*μ_{2221} + 

### Setup MPO

In [8]:
am_pot = AssignManager(pot_sop)
am_pot.assign()
display(*am_pot.Wsym)
W_prod = sympy.Mul(*am_pot.Wsym)
print(*[f"W{i}" for i in range(am_pot.ndim)], "=")
display(W_prod[0].expand())
pot_mpo = am_pot.numerical_mpo(subs=subs)

2025-04-30 12:30:22.613 | INFO     | pympo.bipartite:assign:286 - assigned 1/3
2025-04-30 12:30:22.634 | INFO     | pympo.bipartite:assign:286 - assigned 2/3
2025-04-30 12:30:22.640 | INFO     | pympo.bipartite:assign:286 - assigned 3/3


Matrix([[q^1_1, q^0_1, q^3_1, q^2_1]])

Matrix([
[                                                                                                                            (1.0*E_0*μ_{130} + 1.0*E_1*μ_{131} + 1.0*E_2*μ_{132})*q^0_2,                                                               (0.5*E_0*μ_{1330} + 0.5*E_1*μ_{1331} + 0.5*E_2*μ_{1332})*q^0_2,                                                                                                   (1.0*E_0*μ_{10} + 1.0*E_1*μ_{11} + 1.0*E_2*μ_{12})*q^0_2 + (1.0*E_0*μ_{120} + 1.0*E_1*μ_{121} + 1.0*E_2*μ_{122})*q^1_2 + (0.5*E_0*μ_{1220} + 0.5*E_1*μ_{1221} + 0.5*E_2*μ_{1222})*q^2_2,     0],
[(0.5*E_0*μ_{2230} + 0.5*E_1*μ_{2231} + 0.5*E_2*μ_{2232})*q^2_2 + (1.0*E_0*μ_{230} + 1.0*E_1*μ_{231} + 1.0*E_2*μ_{232})*q^1_2 + (1.0*E_0*μ_{30} + 1.0*E_1*μ_{31} + 1.0*E_2*μ_{32})*q^0_2, (0.5*E_0*μ_{2330} + 0.5*E_1*μ_{2331} + 0.5*E_2*μ_{2332})*q^1_2 + (0.5*E_0*μ_{330} + 0.5*E_1*μ_{331} + 0.5*E_2*μ_{332})*q^0_2, (1.0*E_0*μ_{0} + 1.0*E_1*μ_{1} + 1.0*E_2*μ_{2})*q^0_2 + (1.0*E_0*μ_{20} + 1.

Matrix([
[                                                                                                   q^1_3],
[                                                                                                   q^2_3],
[                                                                                                   q^0_3],
[(0.166666666666667*E_0*μ_{3330} + 0.166666666666667*E_1*μ_{3331} + 0.166666666666667*E_2*μ_{3332})*q^3_3]])

W0 W1 W2 =


1.0*E_0*μ_{0}*q^0_1*q^0_2*q^0_3 + 1.0*E_0*μ_{10}*q^1_1*q^0_2*q^0_3 + 0.5*E_0*μ_{110}*q^2_1*q^0_2*q^0_3 + 0.166666666666667*E_0*μ_{1110}*q^3_1*q^0_2*q^0_3 + 0.5*E_0*μ_{1120}*q^2_1*q^1_2*q^0_3 + 0.5*E_0*μ_{1130}*q^2_1*q^0_2*q^1_3 + 1.0*E_0*μ_{120}*q^1_1*q^1_2*q^0_3 + 0.5*E_0*μ_{1220}*q^1_1*q^2_2*q^0_3 + 1.0*E_0*μ_{130}*q^1_1*q^0_2*q^1_3 + 0.5*E_0*μ_{1330}*q^1_1*q^0_2*q^2_3 + 1.0*E_0*μ_{20}*q^0_1*q^1_2*q^0_3 + 0.5*E_0*μ_{220}*q^0_1*q^2_2*q^0_3 + 0.166666666666667*E_0*μ_{2220}*q^0_1*q^3_2*q^0_3 + 0.5*E_0*μ_{2230}*q^0_1*q^2_2*q^1_3 + 1.0*E_0*μ_{230}*q^0_1*q^1_2*q^1_3 + 0.5*E_0*μ_{2330}*q^0_1*q^1_2*q^2_3 + 1.0*E_0*μ_{30}*q^0_1*q^0_2*q^1_3 + 0.5*E_0*μ_{330}*q^0_1*q^0_2*q^2_3 + 0.166666666666667*E_0*μ_{3330}*q^0_1*q^0_2*q^3_3 + 0.166666666666667*E_1*μ_{1111}*q^3_1*q^0_2*q^0_3 + 0.5*E_1*μ_{111}*q^2_1*q^0_2*q^0_3 + 0.5*E_1*μ_{1121}*q^2_1*q^1_2*q^0_3 + 0.5*E_1*μ_{1131}*q^2_1*q^0_2*q^1_3 + 1.0*E_1*μ_{11}*q^1_1*q^0_2*q^0_3 + 1.0*E_1*μ_{121}*q^1_1*q^1_2*q^0_3 + 0.5*E_1*μ_{1221}*q^1_1*q^2_2*q^0_3 + 1

### Setup Operator

In [9]:
potential = [
    [
        {
            (tuple((i) for i in range(0, ndim))): TensorOperator(
                mpo=pot_mpo
            )
        }
    ]
]

H = TensorHamiltonian(
    ndof=len(basis), potential=potential, kinetic=None, backend=backend
)

operators = {"hamiltonian": H}

## 4. Set wave function (MPS) and All Model

- `m_aux_max` is MPS bond dimenison (maximum of auxiliary index $\tau_p$)

In [10]:
model = Model(basinfo=basinfo, operators=operators)
model.m_aux_max = 9

## 5. Execute Calculation

In [11]:
! ls wf_h2o_polynomial_gs.pkl

wf_h2o_polynomial_gs.pkl


/usr/local/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


F.Y.I., See also about [Simulator](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.simulator_cls.Simulator)

This run type prepare $|\Psi_{\mu}\rangle$ by variationally optimizing
$$\langle \delta\Psi_{\mu}|\Psi_{\mu} - \hat{\mu}\Psi_{\mathrm{gs}}\rangle=0$$
where $\Psi_{\mathrm{gs}}$ is a vibrational ground state wavefunction.

**NOTE**

- Runtype cannnot rebind. If you want to change runtype, you should restart kernel.

In [12]:
jobname = "h2o_polynomial"
simulator = Simulator(jobname, model, backend="numpy", verbose=4)
simulator.operate(
    loadfile_ext="_gs", savefile_ext="_dipole", restart=True, maxstep=10
)

2025-04-30 12:30:25,237 - INFO:main.pytdscf._const_cls - 
     ____     __________   .____ ____   _____
    / _  |   /__  __/ _ \ / ___ / _  \ / ___/
   / /_) /_  __/ / / / ||/ /__ / / )_// /__
  /  ___/ / / / / / / / |.__  / |  __/ ___/
 /  /  / /_/ / / / /_/ /___/ /| \_/ / /
/__/   \__, /_/ /_____/_____/ \____/_/
      /____/

2025-04-30 12:30:25,239 - INFO:main.pytdscf._const_cls - Log file is ./h2o_polynomial_operate/main.log
2025-04-30 12:30:25,240 - INFO:main.pytdscf.simulator_cls - Set integral of DVR basis
2025-04-30 12:30:25,260 - INFO:main.pytdscf.simulator_cls - Set initial wave function (DVR basis)
2025-04-30 12:30:25,280 - INFO:main.pytdscf.simulator_cls - Wave function is loaded from wf_h2o_polynomial_gs.pkl
2025-04-30 12:30:25,282 - INFO:main.pytdscf.simulator_cls - Start: apply operator to wave function
2025-04-30 12:30:25,908 - INFO:main.pytdscf.simulator_cls - Wave function is saved in wf_h2o_polynomial_dipole.pkl
2025-04-30 12:30:25,910 - INFO:main.pytdscf.simulator_

(1.9386775177227347, <pytdscf.wavefunction.WFunc at 0x11bc674d0>)

## 6. Check Log file
See `h2o_polynomial_operate/main.log`, which is defined by `jobname`.

In [13]:
!tail h2o_polynomial_operate/main.log

Wave function is loaded from wf_h2o_polynomial_gs.pkl
Start: apply operator to wave function
----------------------------------------
iterations: 0 norm: 1.9386775177227338
convergence : 0.997739253256778
----------------------------------------
iterations: 1 norm: 1.9386775177227347
convergence : 1.0000000000000007
Wave function is saved in wf_h2o_polynomial_dipole.pkl
End  : apply operator to wave function
